In [1]:
from icpm_experiments import *
import os
import re

In [88]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
# pr-3-11-1151-A48_m37_l3_noise.csv          pr-3-11-1151-A48.txt
# pr-3-11-1151-A48_m50_l4_noise.csv          pr-3-11-1151-A48.txt
# pr-8-11-1912-A57_m39_l3_noise.csv          pr-8-11-1912-A57.txt
# pr-8-11-1912-A57_m52_l4_noise.csv          pr-8-11-1912-A57.txt
# pr-1-11-1244-A59_m41_l3_noise.csv          pr-1-11-1244-A59.txt  
# pr-1-11-1244-A59_m55_l4_noise.csv          pr-1-11-1244-A59.txt


df_name = 'bpi_2019.csv'
data_path = '/home/dsi/user-name/Datasets/'
model_output_path = '/home/dsi/user-name/DynamicConLES/'

base_match = re.match(r'(pr-\d+-\d+-\d+-A\d+)_m\d+_l\d+_noise\.csv', df_name)
base_filename = base_match.group(1) if base_match else None
subfolder = 'bpi'

csv_full_path = os.path.join(data_path, subfolder, df_name)

print("CSV file:", csv_full_path)
print("Model output path:", model_output_path)



common_params = {
    'df_name': df_name,
    'n_train_traces': 10,
    'n_test_traces': None,
    'window_overlap': 0,
    'window_lengths_lst': [15],
    'n_final_markings_lst': [1],
    'min_len': 0,
    'max_len': 79,
    'use_heuristics': True,
    'read_model_from_file': False,
    'return_model': True,
    'data_path': data_path,
    'subfolder': subfolder,
    'print_dataset_stats': True,
    'return_test_df': True,
    'portion': 0.2,
    'test_cases': None,
    'max_successive_merges': 2,
    'random_seed': 42
}

variations = [
    {'use_memo': False, 'nonsync_density_tolerance': 0.1, 'window_lengths_lst': [80]},  
    {'use_memo': False, 'nonsync_density_tolerance': 0.1, 'window_lengths_lst': [10]},
    {'use_memo': True, 'nonsync_density_tolerance': 0.1, 'window_lengths_lst': [10]},
    {'use_memo': False, 'nonsync_density_tolerance': 100, 'window_lengths_lst': [10], 'use_heuristics': False},
]

CSV file: /home/dsi/eli-bogdanov/Datasets/bpi/bpi_2019.csv
Model output path: /home/dsi/eli-bogdanov/DynamicConLES/


In [113]:
# Store results
results = []

for idx, var in enumerate(variations, 1):
    print(f"\nRunning variation {idx}: {var}")
    params = {**common_params, **var}
    res, model, alignments, test_df = compare_window_based_baselines(**params)
    results.append((res, model, alignments, test_df))
    print(f"Variation {idx} done.\n")


Running variation 1: {'use_memo': False, 'nonsync_density_tolerance': 0.1, 'window_lengths_lst': [10]}
Training Dataset Statistics:
Number of traces: 10
Number of unique activities: 10
Number of unique trace variants: 6

Trace Length Statistics:
Average length: 11.70
Min length: 5
Max length: 69

Test Dataset Statistics:
Number of traces: 250828
Number of unique activities: 42
Number of unique trace variants: 11077

Trace Length Statistics:
Average length: 5.66
Min length: 1
Max length: 79

Starting to compute mandatory transitions...
Mandatory transitions computed in 0.4660 seconds
Starting to compute alive transitions...
Alive transitions computed in 0.1162 seconds
Evaluating variant: n_markings=1, window_len=10
Computing trace 8122/11077

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [1]:
# Extract result DataFrames
res_dice, res_dice_m, res_conles = [r[0] for r in results]

# Calculate average cost (2nd column)
avg_cost_dice    = res_dice.iloc[:, 1].mean()
avg_cost_dice_m  = res_dice_m.iloc[:, 1].mean()
avg_cost_conles  = res_conles.iloc[:, 1].mean()

# Calculate average time (3rd column)
avg_time_dice    = res_dice.iloc[:, 2].mean()
avg_time_dice_m  = res_dice_m.iloc[:, 2].mean()
avg_time_conles  = res_conles.iloc[:, 2].mean()

# Calculate the percentage difference in cost (how much higher CONLES is vs DICE)
cost_increase_pct = ((avg_cost_conles - avg_cost_dice) / avg_cost_dice) * 100

# Function to get percent of same cost values between two DataFrames
def percentage_same_cost(df1, df2):
    if len(df1) != len(df2):
        raise ValueError("Results are not the same length—cannot compare!")
    return (df1.iloc[:, 1] == df2.iloc[:, 1]).mean() * 100

# Print results
print(f"Avg cost DICE: {avg_cost_dice:.3f}")
print(f"Avg cost DICE-M: {avg_cost_dice_m:.3f}")
print(f"Avg cost CW: {avg_cost_conles:.3f}")
print(f"CONLES cost is {cost_increase_pct:.2f}% higher than DICE\n")

print(f"Avg time DICE: {avg_time_dice:.4f}")
print(f"Avg time DICE-M: {avg_time_dice_m:.4f}")
print(f"Avg time CW: {avg_time_conles:.4f}\n")

perc_same = percentage_same_cost(res_dice, res_conles)
print(f"Percentage of same costs between DICE and CW: {perc_same:.2f}%")

NameError: name 'results' is not defined